In [85]:
import sqlalchemy
import pandas as pd


In [86]:
db_connection_string = 'sqlite:///./Resources/products.db'
engine  = sqlalchemy.create_engine(db_connection_string)

inspector = sqlalchemy.inspect(engine)
table_names = inspector.get_table_names()
print(table_names)

['Indicators', 'MARA_1_Day_Candles', 'MARA_1_Min_Candles', 'MARA_Info', 'RIOT_1_Day_Candles', 'RIOT_1_Min_Candles', 'RIOT_Info', 'TSLA_1_Day_Candles', 'TSLA_1_Min_Candles', 'TSLA_Info', 'WHD_1_Day_Candles', 'WHD_1_Min_Candles', 'WHD_Info']


In [87]:
indicators_df = pd.read_sql_table('Indicators', con=engine, index_col='Datetime')

In [88]:
print(indicators_df.head())

                      Open   High    Low  Close  Volume  CDLDOJI  \
Datetime                                                           
2021-09-07 03:22:00  35.65  35.69  35.65  35.69     727        0   
2021-09-07 03:23:00  35.69  35.69  35.69  35.69     242      100   
2021-09-07 03:24:00  35.60  35.60  35.60  35.60    1500      100   
2021-09-07 03:26:00  35.60  35.66  35.60  35.66    1410        0   
2021-09-07 03:27:00  35.60  35.60  35.60  35.60     240      100   

                     Trailing Stop  Trade Signal  Pct Change  Stop Loss  \
Datetime                                                                  
2021-09-07 03:22:00      35.461429           0.0    0.000000   0.000000   
2021-09-07 03:23:00      35.477755           0.0    0.000000   0.000000   
2021-09-07 03:24:00      35.386844           0.0   -0.002522   0.000000   
2021-09-07 03:26:00      35.451355           0.0    0.001685  35.451355   
2021-09-07 03:27:00      35.395544           0.0   -0.001683   0.000000  

In [89]:
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import RandomOverSampler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.metrics import balanced_accuracy_score
from imblearn.metrics import classification_report_imbalanced

In [90]:
#Segment features from the trade signal
df = indicators_df.copy()
# X = df.drop(columns=['Trade Signal', 'Open', 'High', 'Low', 'Trailing Stop', 'Pct Change'])
X = df.drop(columns=['Trade Signal'])
y = df["Trade Signal"]

In [91]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, stratify=y)

In [92]:
# import StandardScaler
from sklearn.preprocessing import StandardScaler

# Scale the data
scaler = StandardScaler()
X_scaler = scaler.fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [93]:
display(X_train_scaled, y_train)

array([[ 0.84681157,  0.84903809,  0.85687467, ...,  0.14231127,
         1.25242134,  0.        ],
       [ 1.25673297,  1.24442434,  1.25024479, ..., -0.41218489,
        -0.88334066,  0.        ],
       [ 1.29486519,  1.28729755,  1.28838978, ..., -0.41073832,
        -0.88334066,  0.        ],
       ...,
       [-1.38392349, -1.37560501, -1.3722227 , ...,  0.33508014,
         0.92353336,  0.        ],
       [-1.44112183, -1.45182404, -1.43897641, ..., -0.5029742 ,
        -0.88334066,  0.        ],
       [-1.22186154, -1.23269431, -1.22441089, ...,  0.16605576,
         0.94287048,  0.        ]])

Datetime
2021-09-08 14:08:00    0.0
2021-09-09 10:41:00    0.0
2021-09-09 10:31:00    0.0
2021-09-23 03:01:00    0.0
2021-09-29 07:19:00    0.0
                      ... 
2021-09-28 10:16:00    0.0
2021-09-23 13:18:00    0.0
2021-09-30 12:37:00    0.0
2021-09-30 09:29:00    0.0
2021-10-01 11:11:00    0.0
Name: Trade Signal, Length: 7968, dtype: float64

In [94]:
# MODEL
from sklearn.svm import SVC
# from sklearn.svm import SVR

# Instantiate a linear SVM model
model = SVC(kernel='linear')
# model = LogisticRegression()
model

SVC(kernel='linear')

In [95]:
# FIT
# Fit the data
model.fit(X_train_scaled, y_train)

ValueError: The number of classes has to be greater than one; got 1 class

In [ ]:
# PREDICT
# Make predictions using the test data
y_pred = model.predict(X_test_scaled)

results = pd.DataFrame(
    {
        "Predictions": y_pred,
        "Actual":y_test
    }
)
results

,Predictions,Actual
Datetime,,
2021-09-23 12:12:00,1.0,1.0
2021-10-01 14:56:00,0.0,0.0
2021-09-09 13:58:00,0.0,1.0
2021-09-28 13:23:00,0.0,0.0
2021-09-27 08:37:00,1.0,1.0
...,...,...
2021-09-23 08:37:00,1.0,1.0
2021-09-09 15:27:00,1.0,1.0
2021-10-01 13:45:00,1.0,1.0


In [ ]:
# EVALUATE
# Generate confusion matrix
print(confusion_matrix(y_test, y_pred))

[[ 802  292]
 [ 236 1325]]


In [ ]:
# Generate Classification Report
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

         0.0       0.77      0.73      0.75      1094
         1.0       0.82      0.85      0.83      1561

    accuracy                           0.80      2655
   macro avg       0.80      0.79      0.79      2655
weighted avg       0.80      0.80      0.80      2655

